In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
os.environ['GOOGLE_API_KEY']=os.getenv('GOOGLE_API_KEY')
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRAINCING_V2']="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')

d:\sudesh\Gerative AI\Genrative AI langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#data ingestion technique 
from langchain_community.document_loaders import WebBaseLoader

In [3]:
loader=WebBaseLoader("https://python.langchain.com/docs/tutorials/")

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='\n\n\n\n\nTutorials | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL da

### Steps to be follow
load Document --> Docs--> Divide docs into chunks --> text--> vectors--> Vector Embbedings--> Vector StoreDB

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_split=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [6]:
documents=text_split.split_documents(docs)

In [7]:
from langchain_community.embeddings import OllamaEmbeddings
embedings=OllamaEmbeddings(model="gemma:2b")

C:\Users\Komal Patil\AppData\Local\Temp\ipykernel_5560\3554571036.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedings=OllamaEmbeddings(model="gemma:2b")


In [8]:
##vector store DB
from langchain_community.vectorstores import FAISS
vectorstoreDB=FAISS.from_documents(documents,embedings)

In [10]:
vectorstoreDB

In [11]:
# now we search the contents
query="LangSmith allows you to closely trace, monitor and evaluate your LLM application."
result=vectorstoreDB.similarity_search(query=query)
result[0].page_content

'Chat models and prompts: Build a simple LLM application with prompt templates and chat models.\nSemantic search: Build a semantic search engine over a PDF with document loaders, embedding models, and vector stores.\nClassification: Classify text into categories or labels using chat models with structured outputs.\nExtraction: Extract structured data from text and other unstructured media using chat models and few-shot examples.\n\nRefer to the how-to guides for more detail on using all LangChain components.\nOrchestration\u200b\nGet started using LangGraph to assemble LangChain components into full-featured applications.'

In [13]:
# define llm
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro")
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002566C4D6710>, default_metadata=())

In [14]:
# Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
"""
Answer the question based on provided context.
<context>
{context}
<context>
"""
)
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the question based on provided context.\n<context>\n{context}\n<context>\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002566C4D6710>, default_metadata=())
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "LangSmith allows you to closely trace, monitor and evaluate your LLM application.",
    "context": [
        Document(
            page_content="LangSmith allows you to closely trace, monitor and evaluate your LLM application. It seamlessly integrates with LangChain, and you can use it to inspect and debug individual steps of your chains as you build."
        )
    ]
})


'LangSmith helps developers trace, monitor, and evaluate LLM applications, particularly those built using LangChain, by allowing inspection and debugging of individual chain steps.'

In [18]:
# input-->Retriver-->vectorstoredb
vectorstoreDB
retriver=vectorstoreDB.as_retriever()
from langchain.chains import create_retrieval_chain
retriver_chian=create_retrieval_chain(retriver,document_chain)


In [19]:
retriver_chian

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025631E664A0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the question based on provided context.\n<context>\n{context}\n<context>\n'), additional_kwargs={})])
            | ChatGoogleGenerativeAI(m

In [20]:
# # get the response from retriver
response=retriver_chian.invoke({"input":"LangSmith allows you to closely trace, monitor and evaluate your LLM application."})
response['answer']

'This document provides a brief overview of various functionalities offered by LangChain, including:\n\n* **Building LLM Applications:** Using prompt templates and chat models.\n* **Semantic Search:** Creating a search engine for PDFs using document loaders, embeddings, and vector stores.\n* **Classification:** Categorizing text using chat models with structured outputs.\n* **Extraction:** Extracting structured data from unstructured sources like text using chat models and few-shot examples.\n* **Orchestration:** Building complex applications by combining LangChain components with LangGraph.\n* **Specific Applications:** Building chatbots, agents, RAG pipelines, question-answering systems (with SQL and Graph databases), and summarization tools.\n\nIt also lists several "how-to" topics related to various aspects of LangChain usage, such as query analysis, caching, callbacks, handling rate limits, and working with different models and tools.  It encourages users to refer to specific guid

In [21]:
response

{'input': 'LangSmith allows you to closely trace, monitor and evaluate your LLM application.',
 'context': [Document(id='579f43ba-09ef-4cb3-8745-90bee046ae43', metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='Chat models and prompts: Build a simple LLM application with prompt templates and chat models.\nSemantic search: Build a semantic search engine over a PDF with document loaders, embedding models, and vector stores.\nClassification: Classify text into categories or labels using chat models with structured outputs.\nExtraction: Extract structured data from text and other unstructured media using chat models and few-shot examples.\n\nRefer to the how-to guides for more detail on using all LangChain components.\nOrchestration\u200b\nGet started u

In [22]:
response['context']

[Document(id='579f43ba-09ef-4cb3-8745-90bee046ae43', metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='Chat models and prompts: Build a simple LLM application with prompt templates and chat models.\nSemantic search: Build a semantic search engine over a PDF with document loaders, embedding models, and vector stores.\nClassification: Classify text into categories or labels using chat models with structured outputs.\nExtraction: Extract structured data from text and other unstructured media using chat models and few-shot examples.\n\nRefer to the how-to guides for more detail on using all LangChain components.\nOrchestration\u200b\nGet started using LangGraph to assemble LangChain components into full-featured applications.'),
 Document(id='eee07488-